- Mục tiêu : Crawl toàn bộ giá của tất cả mã cổ phiếu trong lịch sử từ website được cty VNDIRECT lưu trữ sử dụng API

In [17]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd
import json

def convert_to_datetime(string):
    format = "%Y-%m-%d"
    return datetime.strptime(string,format)


def craw_data_vndirect(str_start_date):
    url = "https://finfo-api.vndirect.com.vn/v4/stock_prices"
    HEADERS = {'content-type': 'application/x-www-form-urlencoded', 'User-Agent': 'Mozilla'}
    
    start_date = convert_to_datetime(str_start_date)
    destination_date = start_date + relativedelta(months= 12)
    str_destination_date = destination_date.strftime("%Y-%m-%d")
    
    query = '~date:gte:' + str_start_date + '~date:lte:' + str_destination_date
    param_1 = {"q" : query , 'size': 5}
    response_1 = requests.get(url, params = param_1, headers = HEADERS)
    
    totalElements =  response_1.json()['totalElements']
    totalPages = (totalElements // 200000) + 1
    
    list_temp = []
    for i in range(1,totalPages +1):
        params_2 = {"q" : query , 'size': 200000, 'page': i}
        response_2 = requests.get(url,params = params_2, headers = HEADERS)
        df = pd.DataFrame(response_2.json()['data'])
        list_temp.append(df)
    return pd.concat(list_temp, ignore_index = True)

list_result = []
for i in range(2013, 2023):
    list_result.append(craw_data_vndirect(f"{i}-01-01"))
    print('complete năm: ', i)
print('successfull')

complete năm:  2013
complete năm:  2014
complete năm:  2015
complete năm:  2016
complete năm:  2017
complete năm:  2018
complete năm:  2019
complete năm:  2020
complete năm:  2021
complete năm:  2022
successfull


In [18]:
df_result = pd.concat(list_result,ignore_index = True)
print(df_result.shape)
df_result.head()

(3079217, 25)


,code,date,time,floor,type,basicPrice,ceilingPrice,floorPrice,open,high,...,adLow,adClose,adAverage,nmVolume,nmValue,ptVolume,ptValue,change,adChange,pctChange
0,MHL,2013-04-17,14:04:10,HNX,STOCK,9.3,10.2,8.4,9.3,9.3,...,6.742,6.742,6.742,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000
1,CMI,2013-07-15,14:07:08,HNX,STOCK,4.6,5.0,4.2,4.4,4.7,...,4.154,4.437,4.437,34500.0,158580000.0,0.0,0.0,0.1,0.0944,2.1739
2,MIC,2013-04-17,14:04:10,HNX,STOCK,14.2,15.6,12.8,14.0,14.7,...,14.000,14.700,14.700,1100.0,15470000.0,0.0,0.0,0.5,0.5000,3.5211
3,SFC,2013-06-12,14:06:04,HOSE,STOCK,22.0,23.5,20.5,22.0,22.0,...,9.992,9.992,0.000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000
4,SGT,2013-06-12,14:06:04,HOSE,STOCK,3.4,3.6,3.2,3.2,3.2,...,3.200,3.200,3.200,200.0,640000.0,0.0,0.0,-0.2,-0.2000,-5.8824


In [19]:
df_result.duplicated().value_counts()

False    3075268
True        3949
dtype: int64

In [20]:
data_vndirect = df_result.drop_duplicates()
data_vndirect.shape

(3075268, 25)

In [21]:
data_vndirect.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3075268 entries, 0 to 3079216
Data columns (total 25 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   code          3075268 non-null  object 
 1   date          3075268 non-null  object 
 2   time          3075268 non-null  object 
 3   floor         3075268 non-null  object 
 4   type          3075268 non-null  object 
 5   basicPrice    3075268 non-null  float64
 6   ceilingPrice  3075268 non-null  float64
 7   floorPrice    3075268 non-null  float64
 8   open          3075268 non-null  float64
 9   high          3075268 non-null  float64
 10  low           3075268 non-null  float64
 11  close         3075268 non-null  float64
 12  average       3075268 non-null  float64
 13  adOpen        3075268 non-null  float64
 14  adHigh        3075268 non-null  float64
 15  adLow         3075268 non-null  float64
 16  adClose       3075268 non-null  float64
 17  adAverage     3075268 non-n

In [22]:
#cuối cùng convert sang file csv để lưu trữ
data_vndirect.to_csv(r'C:\Users\daodq\Desktop\Đồ án cuối khóa\vndirect.csv', index =False)
print('complete')

complete


In [2]:
import pandas as pd
df = pd.read_csv("vndirect.csv")
df.head()

,code,date,time,floor,type,basicPrice,ceilingPrice,floorPrice,open,high,...,adLow,adClose,adAverage,nmVolume,nmValue,ptVolume,ptValue,change,adChange,pctChange
0,MHL,2013-04-17,14:04:10,HNX,STOCK,9.3,10.2,8.4,9.3,9.3,...,6.742,6.742,6.742,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000
1,CMI,2013-07-15,14:07:08,HNX,STOCK,4.6,5.0,4.2,4.4,4.7,...,4.154,4.437,4.437,34500.0,158580000.0,0.0,0.0,0.1,0.0944,2.1739
2,MIC,2013-04-17,14:04:10,HNX,STOCK,14.2,15.6,12.8,14.0,14.7,...,14.000,14.700,14.700,1100.0,15470000.0,0.0,0.0,0.5,0.5000,3.5211
3,SFC,2013-06-12,14:06:04,HOSE,STOCK,22.0,23.5,20.5,22.0,22.0,...,9.992,9.992,0.000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000
4,SGT,2013-06-12,14:06:04,HOSE,STOCK,3.4,3.6,3.2,3.2,3.2,...,3.200,3.200,3.200,200.0,640000.0,0.0,0.0,-0.2,-0.2000,-5.8824
